<div style='background-image: url("https://www.geophysik.uni-muenchen.de/ROMY/theme/images/ringlaser_wideformat.jpg") ; padding: 0px ; background-size: cover ; border-radius: 15px ; height: 250px; background-position: 0% 80%'>
    <div style="float: left ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.9) ; width: 35% ; height: 150px">
        <div style="position: relative ; top: 50%;  transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.9) ; line-height: 100%"> <br>ROMY - <br><br>  Synthetic Sagnac Processing </div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.7)"></div>
        </div>
    </div>

## Import 

### Official Libraries

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import time
import pickle

from scipy.signal import resample, hilbert, correlate, decimate, butter, spectrogram, sosfilt, filtfilt, iirnotch
from tqdm import tqdm
from obspy import UTCDateTime, read, read_inventory, Trace
import obspy

### Own Processing Methods

In [2]:
import sys
sys.path.insert(0, 'submodules')

In [3]:
from EchoPerformance import __echo_performance
from CreateSyntheticEventV2 import __create_synthetic_event_v2
from MinimizeResidual import __minimize_residual
from CreateLinearChirp import __create_linear_chirp
from Tapering import __tapering
from InstaFreqHilbert import __insta_freq_hilbert
from Normalize import __normalize
from WriteToMseed import __write_to_mseed
from LoadMseed import __load_mseed
from Modulation import __modulation
from QuerrySeismoData import __querry_seismo_data
from RingLaser import RingLaser
from datetime import datetime


In [4]:
# def third_order_derive(x, pos="MIDDLE"):

#     coeff = np.zeros((5,5))
#     norm = np.array([0,0])
#     sum = 0.

#     norm[0] = 12.;
#     norm[1] = 84.;

#     coeff[0][0] = -125.;
#     coeff[0][1] =  136.;
#     coeff[0][2] =   48.;
#     coeff[0][3] =  -88.;
#     coeff[0][4] =   29.;
#     coeff[1][0] =  -38.;
#     coeff[1][1] =   -2.;
#     coeff[1][2] =   24.;
#     coeff[1][3] =   26.;
#     coeff[1][4] =  -10.;
#     coeff[2][0] =    1.;
#     coeff[2][1] =   -8.;
#     coeff[2][2] =    0.;
#     coeff[2][3] =    8.;
#     coeff[2][4] =   -1.;
#     coeff[3][0] =   10.;
#     coeff[3][1] =  -26.;
#     coeff[3][2] =  -24.;
#     coeff[3][3] =    2.;
#     coeff[3][4] =   38.;
#     coeff[4][0] =  -29.;
#     coeff[4][1] =   88.;
#     coeff[4][2] =  -48.;
#     coeff[4][3] = -136.;
#     coeff[4][4] =  125.;

#     for i in range(0, 5):
#         sum += coeff[pos][i]*x[i];
    
#     if pos == 'MIDDLE':
#         sum /= norm[0];
    
#     else:
#         sum /= norm[1];


#     return sum

In [5]:
from RingLaser import RingLaser

## define G-Ring
G = RingLaser(side=4., form="square", wl=632.8e-9, lat=49.16)

## define ROMY Z-Ring
ROMY_Z = RingLaser(side=11., form="triangle", wl=632.8e-9, lat=49.16)


### Own Plotting Methods

In [6]:
from MakeplotTraceAndSpectrum import __makeplot_trace_and_spectrum
from MakeplotTraceAndPSD import __makeplot_trace_and_psd
from MakeplotModulatedSignal import __makeplot_modulated_signal
from MakeplotDemodulationQuality import __makeplot_demodulation_quality


## Configurations

In [7]:
## define configuration class 
class configuration():

    def __init__(self):
        
        self.T = None
        self.sps = None
        self.sgnc = None

    def show(self):
        print(self.T, self.sps, )

        

## initalize new object
conf = configuration()






In [26]:
path2plots="/home/andbro/Documents/ROMY/SagnacProcessing/plots/V3/"

save = True


conf.T = 1400

conf.sps = 5000

conf.sgnc = ROMY_Z.get_sagnac_frequency()


# modulation_index = ROMY_Z.get_scale_factor()

# modulation_scheme = 3


conf.win_type = "flanks"
conf.win_perc = 5


conf.upsampling_factor = 2

conf.sgnc_is = 553.35

conf.f_lower_bp, f_upper_bp = conf.sgnc_is-5, conf.sgnc_is+5

conf.f_upper_lp = 200

conf.signal = 'real'  ## 'synthetic'  or 'real'


conf.Twindow = 200 # seconds


conf.taper = False

conf.upsampling = False

conf.bandpass_pre = True
conf.lowpass_post = False

conf.normalize = True
conf.remove_median = True

conf.bools = {'taper':conf.taper,'upsampling':conf.upsampling,'bandpass_pre':conf.bandpass_pre,
         'lowpass_post':conf.lowpass_post, 'normalize':conf.normalize, 'remove_median':conf.remove_median}

## Read synthetic

In [27]:

if conf.signal == 'synthetic':

    name = 'eigenmodes'  # 'model_greece' (greece), 'model_lowperiod' (kermadec 0.001-0.1 Hz), 'car' (1.0-10.0 Hz), 'eigenmodes' (0.003-0.03 Hz)
    
    ifile = open(f'/home/andbro/notebooks/Sagnac_processing/data/{name}.syn',"rb")

    synthetic_signal = pickle.load(ifile)
    timeline = np.arange(0, T+1/sps, 1/sps)
    
    ifile = open(f'/home/andbro/notebooks/Sagnac_processing/data/{name}.mod',"rb")
        
    modeltrace = pickle.load(ifile)
    time_modeltrace = np.arange(0, T+1/sps, 1/sps)
    
    synthetic_signal.size, timeline.size, modeltrace.size, time_modeltrace.size
    

## Read raw real

In [28]:
from andbro__querrySeismoData import __querrySeismoData

## kermadec
# tbeg = UTCDateTime("2021-03-04 18:45")
# tend = UTCDateTime("2021-03-04 18:45")+conf.T

## greece
tbeg = UTCDateTime("2021-03-03 10:18")
tend = UTCDateTime("2021-03-03 10:18")+conf.T


#### real synthetic trace

In [29]:
if conf.signal == 'real':
    
    # st = read('/home/andbro/Documents/ROMY/data/BW.DROMY..FJZ.D.2021.049', 
    #          starttime=UTCDateTime("2021-02-18 15:00"),
    #          endtime=UTCDateTime("2021-02-18 15:00")+T,
    #          )
    
    try:
        st = obspy.read(f"/home/andbro/notebooks/Sagnac_processing/data/event_T{conf.T}.mseed")

    except:
        st, inv = __querrySeismoData(
                                    seed_id='BW.DROMY..FJZ',
                                    starttime=tbeg,
                                    endtime=tend,
                                    where="george",
                                    path=None,
                                    restitute=True,
                                    detail=None,
                                    )

        st.write(f"/home/andbro/notebooks/Sagnac_processing/data/event_T{T}.mseed", format="MSEED")


    tbeg_new = st[0].stats.starttime
    tend_new = st[0].stats.endtime
    print(st)

    ## define synthetic signal
    synthetic_signal = st[0].data; del st

    ## normalized signal
    synthetic_signal = __normalize(synthetic_signal)
    
    ## define time axis for synthetic axis
    timeline = np.linspace(0, conf.T, len(synthetic_signal))


NameError: name 'T' is not defined

#### real model trace 

In [ ]:
if conf.signal == 'real':
    
    # trace = read('/home/andbro/Documents/ROMY/data/ROMY_BJZ_2021-02-18.mseed',
    #             starttime= UTCDateTime("2021-02-18 14:59"), 
    #             endtime=UTCDateTime("2021-02-18 15:00")+T+1
    #             )
    try:
        trace, inv = __querrySeismoData(    
                                        seed_id='BW.ROMY.10.BJZ',
                                        starttime=tbeg-1,
                                        endtime=tend+1,
                                        where="george",
                                        path=None,
                                        restitute=True,
                                        detail=None,
                                        )
    except:
        print('could not load data')
        
    ## resample to sps and remove mean
    trace.detrend('demean')
    trace.resample(conf.sps)

    ## adjust length to match signal
#     trace.trim(starttime=tbeg, endtime=tend+T)
    trace.trim(starttime=tbeg_new, endtime=tend_new)
    print(trace)

    ## define model trace
    modeltrace = trace[0].data; del trace

    ## define time axis for model trace
    time_modeltrace = np.linspace(0, conf.T, len(modeltrace))
   

In [ ]:
__makeplot_trace_and_spectrum(modeltrace, time_modeltrace, fmax=5); 

## Tests

### Bandpass Test

In [ ]:
from RunSagnacProcessing_V3 import __run_sagnac_processing

outs, half_bands = [], []
for half_band in range(1,50,1):
    
    conf.f_lower_bp, conf.f_upper_bp = conf.sgnc_is-half_band, conf.sgnc_is+half_band
    print(half_band)
    
    out = __run_sagnac_processing(
                                conf.sgnc, 
                                conf.T, 
                                conf.sps, 
                                conf.upsampling_factor, 
                                conf.f_lower_bp, 
                                conf.f_upper_bp, 
                                conf.signal,
                                conf.Twindow,
                                conf.win_type, 
                                conf.win_perc,
                                conf.f_upper_lp,
                                modeltrace,
                                time_modeltrace,
                                timeline,
                                synthetic_signal,
                                conf.bools,
                                show=False
                               )
    outs.append(out)
    half_bands.append(half_band)
    
    del out
    

In [ ]:
# half_bands = range(1,25,1)
vals = np.array(np.transpose(outs))
bands = np.array(half_bands)*2

## _______________________________________________________________

fig, ax = plt.subplots(4, 1, figsize=(15, 10), sharex=True)

font = 14


for i in range(len(vals)):
    ax[i].plot(bands, vals[i], zorder=1, color='grey')
    ax[i].scatter(bands, vals[i], zorder=2, color='k')

    ax[i].grid(zorder=0, ls='--')

ax[0].set_ylabel("lag at max. CC (s)", fontsize=font)
ax[1].set_ylabel("rmse trace", fontsize=font)
ax[2].set_ylabel("rmse ASD", fontsize=font)
ax[3].set_ylabel("rmse phase", fontsize=font)

ax[3].set_xlabel(f"frequency band width (Hz) - centered around Sagnac: {round(conf.sgnc,2)} Hz", fontsize=font)

# ax[0].set_title(f"DC Offset Beam 1 and 2 (median = {round(med,2)} V)", fontsize=font+2)


if save:
    datestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    ## save plot
    fig.savefig(path2plots+f"plot__{datestamp}_bandwidth_test_50Hz"+".png", 
                dpi=300, facecolor='w', edgecolor='w',
                orientation='portrait', format="png",
                transparent=False, bbox_inches="tight", pad_inches=0.2,
               )
    

    ## save configurations
    oconfig = f'{path2plots}parameters_{datestamp}.config'

    picklefile = open(oconfig, "wb")
    
    pickle.dump(conf, picklefile); picklefile.close()
    
    
    ## save output as object
    odata = f'{path2plots}output_{datestamp}.data'

    picklefile = open(odata, "wb")
    
    pickle.dump(outs, picklefile); picklefile.close()

### Window Test

In [ ]:
from RunSagnacProcessing_V3 import __run_sagnac_processing

outs, wins = [], []
for win in range(100,1000,100):
    
    conf.Twindow = win 
    
    print(win)
    
    out = __run_sagnac_processing(
                                conf.sgnc, 
                                conf.T, 
                                conf.sps, 
                                conf.upsampling_factor, 
                                conf.f_lower_bp, 
                                conf.f_upper_bp, 
                                conf.signal,
                                conf.Twindow,
                                conf.win_type, 
                                conf.win_perc,
                                conf.f_upper_lp,
                                modeltrace,
                                time_modeltrace,
                                timeline,
                                synthetic_signal,
                                conf.bools,
                                show=False
                               )
    outs.append(out)
    wins.append(win)
    
    del out
    

In [ ]:
vals = np.array(np.transpose(outs))



## _______________________________________________________________

fig, ax = plt.subplots(4, 1, figsize=(15, 10), sharex=True)

font = 14


for i in range(len(vals)):
    ax[i].plot(wins, vals[i], zorder=1, color='grey')
    ax[i].scatter(wins, vals[i], zorder=2, color='k')

    ax[i].grid(zorder=0, ls='--')

ax[0].set_ylabel("lag at max. CC (s)", fontsize=font)
ax[1].set_ylabel("rmse trace", fontsize=font)
ax[2].set_ylabel("rmse ASD", fontsize=font)
ax[3].set_ylabel("rmse phase", fontsize=font)

ax[3].set_xlabel(f"window length (s)", fontsize=font)

# ax[0].set_title(f"DC Offset Beam 1 and 2 (median = {round(med,2)} V)", fontsize=font+2)


if save:
    datestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    ## save plot
    fig.savefig(path2plots+f"plot__{datestamp}_wind_test_100-900"+".png", 
                dpi=300, facecolor='w', edgecolor='w',
                orientation='portrait', format="png",
                transparent=False, bbox_inches="tight", pad_inches=0.2,
               )
    

    ## save configurations
    oconfig = f'{path2plots}parameters_{datestamp}.config'

    picklefile = open(oconfig, "wb")
    
    pickle.dump(conf, picklefile); picklefile.close()
    
    
    ## save output as object
    odata = f'{path2plots}output_{datestamp}.data'

    picklefile = open(odata, "wb")
    
    pickle.dump(outs, picklefile); picklefile.close()